<a href="https://colab.research.google.com/github/soundaraya-01/Fmmllab-2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answer 1:Increasing the percentage of data allocated to the validation set generally provides a more accurate estimate of the model's performance on unseen data, as the validation set becomes more representative of the overall data distribution. However, this comes at the cost of reducing the amount of data available for training, which might lead to a less well-trained model and potentially lower performance.

Conversely, reducing the percentage of data allocated to the validation set means more data is available for training, which can improve the model's ability to learn. However, with less data in the validation set, the performance estimate may be less reliable and more susceptible to fluctuations, potentially leading to an overestimate of the model's generalization capability.

In summary:
- Increasing the validation set size improves the reliability of the performance estimate but reduces training data.
- Decreasing the validation set size provides more training data but can make the performance estimate less reliable.

Answer 2:
          The size of the training and validation sets affects the prediction of accuracy on the test set in the following ways:

1. **Larger Validation Set:**
   - **Advantages:** A larger validation set provides a more accurate estimate of the model's performance and its ability to generalize. This means the validation accuracy is likely to be a better predictor of test set accuracy because the validation set better represents the overall data distribution.
   - **Disadvantages:** However, if the validation set is too large, it reduces the amount of data available for training. This can lead to a less well-trained model, which might negatively affect its performance on both the validation and test sets.

2. **Smaller Validation Set:**
   - **Advantages:** A smaller validation set means more data is available for training, which can improve the model's performance by allowing it to learn better from the available data.
   - **Disadvantages:** With a smaller validation set, the performance estimate is less stable and might not be as reliable. The validation accuracy might not accurately reflect the model’s performance on the test set due to higher variance in the validation estimate.

In summary:
- A larger validation set typically provides a more accurate and stable estimate of the model's performance, leading to better predictions of test set accuracy.
- A smaller validation set provides less reliable performance estimates, which might lead to inaccurate predictions of test set accuracy, though it can improve the model’s training performance.

Answer 3:
          A commonly used practice is to reserve about 10-20% of the dataset for the validation set. This balance allows for a sufficient amount of data to assess model performance reliably while still leaving a substantial portion for training. Here’s a brief guide:

- **10% Validation Set:** Provides a robust estimate of model performance while still retaining most of the data for training. This is often suitable for large datasets where even a small percentage can represent a substantial number of samples.

- **20% Validation Set:** Offers a more comprehensive assessment of model performance, which can be especially useful for smaller datasets where each sample is critical. However, this reduces the amount of data available for training, which might affect the model's ability to learn effectively.

Ultimately, the optimal percentage can vary depending on the dataset size and specific requirements of your model. For smaller datasets, a higher percentage might be necessary to obtain reliable performance estimates, while larger datasets can afford a smaller percentage for validation.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Answer 1:Yes, averaging the validation accuracy across multiple splits typically provides a more consistent and reliable estimate of a model's performance. When you use multiple splits (e.g., through k-fold cross-validation), each split provides a separate evaluation of the model's accuracy. By averaging these evaluations, you reduce the impact of any single split's variability and get a more stable estimate of how the model performs on unseen data. This approach helps to mitigate the effects of any peculiarities or biases present in individual data splits, leading to a more robust measure of the model's generalization capability.

Answer 2:
          Averaging validation accuracy across multiple splits provides a more reliable estimate of the model's performance during training but not necessarily a direct estimate of test accuracy. Here’s how it works:

1. **Validation Accuracy**: This is an estimate of how well the model performs on unseen data during the training phase. Averaging across multiple splits (e.g., using k-fold cross-validation) helps get a more stable and reliable measure of the model's performance during training.

2. **Test Accuracy**: This is the performance measure on a completely independent test set that the model has not seen during training or validation. It provides a true estimate of the model’s ability to generalize to new, unseen data.

While averaging validation accuracy gives a better estimate of how the model might perform in practice compared to a single validation split, it’s still not a direct measure of test accuracy. The best way to get an accurate estimate of test accuracy is to evaluate the model on a separate, hold-out test set after training is complete.

Answer 3:
          Increasing the number of iterations, or in the context of cross-validation, the number of folds, generally improves the reliability of the performance estimate. Here’s how:

1. **Stability and Variance**: With more iterations (e.g., more folds in cross-validation), the estimates of model performance tend to become more stable and less sensitive to the particular splits of the data. This reduces the variance of the performance estimate because each fold provides additional information about the model’s performance on different subsets of the data.

2. **Bias-Variance Trade-off**: More folds in cross-validation can decrease the variance of the performance estimate because each fold is based on a larger portion of the data. However, with very high numbers of folds, the benefit may plateau, and the computational cost increases. The trade-off is that while more folds reduce variance, they can also slightly increase computational expense.

3. **Computational Cost**: More iterations or folds mean more computations, as the model is trained and validated multiple times. For instance, with 10-fold cross-validation, the model is trained and validated 10 times, whereas with 5-fold cross-validation, it's done 5 times.

In summary, increasing the number of iterations generally improves the reliability and stability of the performance estimate, but it’s important to balance this with computational resources and diminishing returns on the estimate’s accuracy.

Answer 4:
          Increasing the number of iterations (e.g., folds in cross-validation) can help mitigate some issues associated with small datasets, but it has limitations and should be used with caution:

1. **Increased Robustness**: With very small datasets, using cross-validation with more folds can provide a more robust estimate of model performance. For instance, in k-fold cross-validation, if the dataset is very small, increasing the number of folds (e.g., using leave-one-out cross-validation) ensures that each data point is used in both training and validation sets, maximizing the use of available data.

2. **Overfitting Concerns**: While more folds can provide a better estimate of model performance, the risk of overfitting still exists. With very small datasets, even leave-one-out cross-validation may not fully address the risk of overfitting because the model can become overly sensitive to the small sample size.

3. **Computational Overhead**: More folds increase the computational cost, as the model has to be trained and validated many times. For very small datasets, this might not be a significant issue, but it's important to consider the computational resources required.

4. **Limited Generalization**: No matter how many iterations or folds you use, the performance estimate will still be constrained by the limited amount of data. The model’s ability to generalize to unseen data may still be limited if the dataset is too small.

In summary, while increasing iterations or folds in cross-validation can help make the most of a small dataset, it cannot entirely overcome the inherent limitations of a very small dataset. Combining this approach with other strategies, such as data augmentation or transfer learning, might be necessary to improve model performance and generalization.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.